## Train a new tokenizer based on existing models 

In [1]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer
import config

In [2]:
# This can take a few minutes to load, so grab a coffee or tea while you wait!
raw_datasets = load_dataset("code_search_net", "python")
raw_datasets["train"]

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/941M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset code_search_net downloaded and prepared to /Users/huang/.cache/huggingface/datasets/code_search_net/python/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
    num_rows: 412178
})

In [3]:
## print one raw data to inspect
print(raw_datasets["train"][123456]["whole_func_string"])

def updater():
    """Update the current installation.

    git clones the latest version and merges it with the current directory.
    """
    print('%s Checking for updates' % run)
    # Changes must be separated by ;
    changes = '''major bug fixes;removed ninja mode;dropped python < 3.2 support;fixed unicode output;proxy support;more intels'''
    latest_commit = requester('https://raw.githubusercontent.com/s0md3v/Photon/master/core/updater.py', host='raw.githubusercontent.com')
    # Just a hack to see if a new version is available
    if changes not in latest_commit:
        changelog = re.search(r"changes = '''(.*?)'''", latest_commit)
        # Splitting the changes to form a list
        changelog = changelog.group(1).split(';')
        print('%s A new version of Photon is available.' % good)
        print('%s Changes:' % info)
        for change in changelog: # print changes
            print('%s>%s %s' % (green, end, change))

        current_path = os.getcwd().split('/') #

### 1) We will have to transform the dataset into an iterator of lists of texts 
- Using a Python generator, we can avoid Python loading anything into memory until it’s actually necessary. To create such a generator, you just to need to replace the brackets with parentheses:

In [4]:
## make it a generator 
def get_training_corpus():
    dataset = raw_datasets["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]
training_corpus = get_training_corpus()

### 2) Train a new Tokenizer 
- Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as GPT-2, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.
- Note that AutoTokenizer.train_new_from_iterator() only works if the tokenizer you are using is a “fast” tokenizer.

In [5]:
# for example, let's try to use gpt2 tokenizer as base 
# this can take a bit of time 
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [10]:
## now we can see an example of the results 
example = """class LinearLayer():
    def __init__(self, input_size, output_size):
        self.weight = torch.randn(input_size, output_size)
        self.bias = torch.zeros(output_size)

    def __call__(self, x):
        return x @ self.weights + self.bias
    """
print(old_tokenizer.tokenize(example))
print(tokenizer.tokenize(example))

## we can see several originally seperated words are now groupped together 

['class', 'ĠLinear', 'Layer', '():', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġdef', 'Ġ__', 'init', '__', '(', 'self', ',', 'Ġinput', '_', 'size', ',', 'Ġoutput', '_', 'size', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'rand', 'n', '(', 'input', '_', 'size', ',', 'Ġoutput', '_', 'size', ')', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġself', '.', 'b', 'ias', 'Ġ=', 'Ġtorch', '.', 'zer', 'os', '(', 'output', '_', 'size', ')', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġdef', 'Ġ__', 'call', '__', '(', 'self', ',', 'Ġx', '):', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'Ġx', 'Ġ@', 'Ġself', '.', 'weights', 'Ġ+', 'Ġself', '.', 'b', 'ias', 'Ċ', 'Ġ', 'Ġ', 'Ġ', 'Ġ']
['class', 'ĠLinear', 'Layer', '():', 'ĊĠĠĠ', 'Ġdef', 'Ġ__', 'init', '__(', 'self', ',', 'Ġinput', '_', 'size', ',', 'Ġoutput', '_', 'size', '):', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'weight', 'Ġ=', 'Ġtorch', '.', 'randn', '(', 'input', '_', 'size', ',', 'Ġoutput', '_', 'size', ')', 'ĊĠĠĠĠĠĠĠ', 'Ġself', '.', 'bias', 'Ġ=', 'Ġtorch'

#### We can compare the vocabular and add additional to original tokenizer follwoing: https://github.com/georgian-io/Transformers-Domain-Adaptation/blob/master/notebooks/GuideToTransformersDomainAdaptation.ipynb
- but with out using their packeage (They have an outdated tokenizer version)
- also in our MLM pretraining notebook, there is a vocabulary expansing step 

In [10]:
new_key = []
for k in tokenizer.vocab.keys():
    if old_tokenizer.vocab.get(k) is None:
        new_key.append(k)
print(len(new_key))

32161


### 4) Save trained tokenizer 

In [17]:
outdir = os.path.join(config.data_folder,'new_tokenizer','code-search-net-tokenizer')
tokenizer.save_pretrained(outdir)

('/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/tokenizer_config.json',
 '/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/special_tokens_map.json',
 '/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/vocab.json',
 '/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/merges.txt',
 '/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/added_tokens.json',
 '/media/chengyu/Elements1/HuggingFace/new_tokenizer/code-search-net-tokenizer/tokenizer.json')

In [18]:
## to load tokenizer 
tokenizer = AutoTokenizer.from_pretrained(outdir)